In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

import openai
import os
# openai.api_key = ""
os.environ['OPENAI_API_KEY'] = ""

In [2]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('data/TextBooks', glob="./DSC140B-Lec01.pdf", show_progress=True) ### many doc loaders
docs = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.89s/it]


In [3]:
# embeddings = OpenAIEmbeddings() ### many embedding functions
embeddings = HuggingFaceInstructEmbeddings(query_instruction="Represent the query for retrieval: ", model_kwargs={'device':'cuda:0'}, encode_kwargs={'batch_size':32})


/home/reventh/anaconda3/envs/lmtutor/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
vectorstore = FAISS.from_documents(documents, embeddings) ### choose from FAISS, Chroma, 
vectorstore.save_local('data/TextBooks/vectorstore_DSC140B-Lec01.vs')
FAISS.load_local("data/TextBooks/vectorstore_DSC140B-Lec01.vs", embeddings=embeddings)

In [7]:
query = "What is ML algorithm?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

An ML algorithm is a set of precise instructions telling the computer how to learn from data.

This is because real world data has “structure”.

An ML algorithm is a set of precise instructions telling the computer how to learn from data.

Spoiler: the algorithms are usually pretty simple. It’s the data that does the real work.

An ML algorithm is a set of precise instructions telling the computer how to learn from data.

Spoiler: the algorithms are usually pretty simple. It’s the data that does the real work.

This is because real world data has “structure”.

Problem: On a scale from 1-10, how happy is this person?

Recall: Least Squares Regression

▶ Example: predict the price of a laptop.

▶ Choose some features:

▶ CPU speed, amount of RAM, weight (kg).

▶ Prediction function (weighted “vote”):

(price) = 𝑤0 + 𝑤1 × (cpu) + 𝑤2 × (ram) + 𝑤3 × (weight)

▶ Learn 𝑤𝑖 by minimizing squared error.

Representations

▶ Computers don’t understand the concept of a

laptop.


In [1]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
llm = HuggingFacePipeline.from_model_id(
                model_id="lmsys/vicuna-7b-v1.5",
                task="text-generation",#"text2text-generation",#,
                model_kwargs={"temperature": 0.7, "max_length": 32, "torch_dtype": torch.float16},
                pipeline_kwargs={'max_new_tokens':32},
                device="cuda:0"
            )
qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), memory=memory, return_source_documents=True)

In [ ]:
query = "What is ML algorithm?"
result = qa({"question": query})

In [ ]:
result

In [ ]:
from model.llm_langchain_tutor import LLMLangChainTutor

In [ ]:
lmtutor = LLMLangChainTutor(openai_key="")
lmtutor.load_vector_store("data/TextBooks/vectorstore_DSC140B-Lec01")
lmtutor.conversational_qa_init()

In [ ]:
lmtutor.conversational_qa('Is there a discussion session this week?')

In [1]:
from model.llm_langchain_tutor import LLMLangChainTutor

In [2]:
lmtutor = LLMLangChainTutor(embedding='instruct_embedding', llm='hf_facebook/bart-large-cnn', device='cuda:6')

/home/yuheng/.conda/envs/lmtutor/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
lmtutor.load_document(doc_path="data/TextBooks", glob='./DSC140B-Lec01.pdf', chunk_size=100, chunk_overlap=10)
lmtutor.generate_vector_store()
# lmtutor.conversational_qa_init()

100%|██████████| 1/1 [00:00<00:00,  1.40it/s]
Created a chunk of size 121, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 132, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100


In [6]:
lmtutor.similarity_search("is there a discussion session?")[0].page_content

'https://www.sdsc.edu/event_items/ 202304-ChatGPT.html\n\nMain Idea'

In [ ]:
lmtutor.conversational_qa('is there a discussion session this week?')